# 1. Data Processing & Feature Engineering

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("behrad3d/nasa-cmaps")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Yoni\.cache\kagglehub\datasets\behrad3d\nasa-cmaps\versions\1


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict


# train1
data = []
data1_path = path + "\\CMaps\\train_FD001.txt"

with open(data1_path, 'r') as f:
    for line in f:
        # print(line.strip().split('\t'))
        row = list(map(float, line.strip().split(' ')))
        data.append(row)

data1 = torch.tensor(data)

max_time_per_unit = defaultdict(int)
for row in data:
    unit_number, time = row[0], row[1]
    max_time_per_unit[unit_number] = max(max_time_per_unit[unit_number], time)

labels1 = []
for row in data:
    unit_number, time = row[0], row[1]
    max_time = max_time_per_unit[unit_number]
    labels1.append(max_time - time)

In [3]:
class EngineDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [4]:
train1_dataset = EngineDataset(data1, labels1)

dataloader = torch.utils.data.DataLoader(train1_dataset, batch_size=100, shuffle=False)


# 2. RUL Prediction Model 

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class RULModel(nn.Module):
    def __init__(self):
        super(RULModel, self).__init__()
        self.fc1 = nn.Linear(26, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x

In [6]:
len(dataloader)


207

In [7]:
from tqdm.notebook import tqdm

model = RULModel()

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a single progress bar for all epochs
with tqdm(total=epochs, desc="Training Progress", unit="epoch") as pbar:
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0

        for data, labels in dataloader:
            data, labels = data.to(device, dtype=torch.float), labels.to(device, dtype=torch.float)

            # Forward pass
            pred = model(data)
            loss = loss_fn(pred.view(-1), labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate loss
            epoch_loss += loss.item()

        # Update progress bar and epoch statistics
        pbar.set_postfix({"Epoch Loss": f"{epoch_loss / len(dataloader):.4f}"})
        pbar.update(1)


Training Progress:   0%|          | 0/10 [00:00<?, ?epoch/s]